In [1]:
# Module Imports
import os
import argparse
import re

import numpy as np
import matplotlib.pyplot as plt
import spiepy
import h5py

import sidpy
import pyUSID as usid
import pycroscopy as px

/Users/sulaymandesai/.pyenv/versions/3.8.3/lib/python3.8/site-packages/pyUSID/viz/__init__.py:16: FutureWarning: Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID
  warn('Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. '


In [2]:
# When creating Python class, input source code here
mtrx2flat_dir = '/Users/sulaymandesai/Documents/GitHub/STM_file_management'
os.chdir(mtrx2flat_dir)
from flatfile_3 import FlatFile

In [4]:
default_dir = '/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData'
os.chdir(default_dir)

In [5]:
class FlatFileTranslator(usid.NumpyTranslator):
    """
    The above definition of the class states that our FlatFileTranslator inherits all the capabilities and
    behaviors of the NumpyTranslator class and builds on top of it.
    For more information on the Numpy translator: 
    https://pycroscopy.github.io/pyUSID/auto_examples/beginner/plot_numpy_translator.html
    """

    def translate(self, input_folder_path, h5_folder_path):
        """
        This function extracts data and metadata from Omicron Scanning Tunnelling Microscope (STM) Flat Files
        and translates this into the Pycroscopy compatible pyUSID format. 
        
        Parameters
        ----------
        input_folder_path : str
            Path to the input data folder containing all the files and their information
        
        h5_folder_path : str
            Path to h5 data folder to store the USID formatted files

        Returns
        -------
        h5_path : str
            Path to the USID h5 output folder
        """
        """
        --------------------------------------------------------------------------------------------
        1. Extracting data and metadata out of the proprietary file
        --------------------------------------------------------------------------------------------
        1.2 Read the contents of the file into memory
        """
        prevdir = os.getcwd()
        os.chdir(input_folder_path)
        
        h5_path_array = []
        
        for file in os.listdir(input_folder_path):
            file_flat = file
            load_file = FlatFile(file_flat)
            d = load_file.getData()
            
            """
            1.3 Extract all experiment and instrument related parameters
            """
            for i in enumerate(d):
                
                raw_data = d[i[0]].data
                metadata = d[i[0]].info
                
                """
                1.4 Prepare the output file path
                """
                folder_path, file_name = os.path.split(file_flat)
                file_name = file_name[:-7] + '_' + metadata['direction']
                h5_path = os.path.join(h5_folder_path, file_name + '.h5')
                
                """
                1.5 Reshape raw_data into USID 2D shape (position x spectral)
                """
                raw_data_2D = np.reshape(raw_data, (raw_data.shape[0] * raw_data.shape[1], 1))
                
                """
                1.6 Extract or generate parameters that define the position and spectral dimensions
                """
                xaxis = metadata['xreal']
                yaxis = metadata['yreal']
                
                xaxis = xaxis/2
                yaxis = yaxis/2
                
                num_rows = int(metadata['yres'])
                num_cols = int(metadata['xres'])
                num_pos = num_rows * num_cols
                
                y_qty = 'Y'
                y_units = 'nm'
                y_vec = np.linspace(-yaxis, yaxis, num_rows, endpoint=True)

                x_qty = 'X'
                x_units = 'nm'
                x_vec = np.linspace(-xaxis, xaxis, num_cols, endpoint=True)
                
                main_data_name = 'STM'
                main_qty = 'Z-height'
                main_units = 'nm'
                
                """
                --------------------------------------------------------------------------------------------
                2. Writing to h5USID file using pyUSID
                --------------------------------------------------------------------------------------------
                2.2 Expressing the Position and Spectroscopic Dimensions using pyUSID.Dimension objects
                """
                pos_dims = [usid.Dimension(x_qty, x_units, x_vec),
                            usid.Dimension(y_qty, y_units, y_vec)]
                
                spec_dims = usid.Dimension(name = 'arb.', units = '', values = int(1))
                
                """
                2.3 Call the translate() function of the base NumpyTranslator class   
                """
                _ = super(FlatFileTranslator, self).translate(h5_path, main_data_name,
                                                     raw_data_2D, main_qty, main_units,
                                                     pos_dims, spec_dims,
                                                     parm_dict=metadata)
                
                h5_path_array.append(h5_path)
        
        # Changing back to original directory
        os.chdir(prevdir)
        
        return h5_path_array          

In [6]:
# instantiate the class first:
my_tran = FlatFileTranslator()

/Users/sulaymandesai/.pyenv/versions/3.8.3/lib/python3.8/site-packages/sidpy/sid/translator.py:42: FutureWarning: Consider using sidpy.Reader instead of sidpy.Translator if possible and contribute your reader to ScopeReaders
  warn('Consider using sidpy.Reader instead of sidpy.Translator if '


In [7]:
help(my_tran)

Help on FlatFileTranslator in module __main__ object:

class FlatFileTranslator(pyUSID.io.numpy_translator.NumpyTranslator)
 |  FlatFileTranslator(*args, **kwargs)
 |  
 |  The above definition of the class states that our FlatFileTranslator inherits all the capabilities and
 |  behaviors of the NumpyTranslator class and builds on top of it.
 |  For more information on the Numpy translator: 
 |  https://pycroscopy.github.io/pyUSID/auto_examples/beginner/plot_numpy_translator.html
 |  
 |  Method resolution order:
 |      FlatFileTranslator
 |      pyUSID.io.numpy_translator.NumpyTranslator
 |      pyUSID.io.numpy_translator.ArrayTranslator
 |      sidpy.sid.translator.Translator
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  translate(self, input_folder_path, h5_folder_path)
 |      This function extracts data and metadata from Omicron Scanning Tunnelling Microscope (STM) Flat Files
 |      and translates this into the Pycroscopy compatible pyUSID format. 
 |      
 | 

In [8]:
input_folder_path = '/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData/STM'
h5_folder_path = '/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData/h5'

In [9]:
# Then call the translate function:
h5_path_2 = my_tran.translate(input_folder_path, h5_folder_path)

/Users/sulaymandesai/.pyenv/versions/3.8.3/lib/python3.8/site-packages/pyUSID/io/dtype_utils.py:62: FutureWarning: pyUSID.io.dtype_utils.contains_integers has been moved to sidpy.base.num_utils.contains_integers. This copy in pyUSID willbe removed in future release. Please update your import statements
  warn('pyUSID.io.dtype_utils.contains_integers has been moved to '
/Users/sulaymandesai/.pyenv/versions/3.8.3/lib/python3.8/site-packages/pyUSID/io/dtype_utils.py:309: FutureWarning: pyUSID.io.dtype_utils.validate_single_string_arg has been moved to sidpy.base.string_utils.validate_single_string_arg. This copy in pyUSID willbe removed in future release. Please update your import statements
  warn('pyUSID.io.dtype_utils.validate_single_string_arg has been moved to '
/Users/sulaymandesai/.pyenv/versions/3.8.3/lib/python3.8/site-packages/pyUSID/io/dtype_utils.py:358: FutureWarning: pyUSID.io.dtype_utils.validate_string_args has been moved to sidpy.base.string_utils.validate_string_args. Th

In [10]:
h5_path_2

['/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData/h5/default_2017Jun09-162147_STM-STM_Spectroscopy--27_1_up-fwd.h5',
 '/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData/h5/default_2017Jun09-162147_STM-STM_Spectroscopy--27_1_up-bwd.h5',
 '/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData/h5/default_2017Jun09-162147_STM-STM_Spectroscopy--27_1_down-fwd.h5',
 '/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData/h5/default_2017Jun09-162147_STM-STM_Spectroscopy--27_1_down-bwd.h5',
 '/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData/h5/default_2017Jun09-162147_STM-STM_Spectroscopy--11_1_up-fwd.h5',
 '/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData/h5/default_2017Jun09-162147_STM-STM_Spectroscopy--11_1_up-bwd.h5',
 '/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData/h5/default_2017Jun09-162147_STM-STM_Spectroscopy--11_1_down-fwd.h5',
 '/Users/sulaymandesai/Documents/Year_4/MSciProject/LoadData/h5/default_2017Jun09-162147_STM-STM_S